# Drug Amount Identification
For my HRP, I will be using AI tools to scrape medical notes to identify drug amounts administered to patients. If I can do this successfully, I will use these methods to flag discrepencies between drugs given to patients according to the prescriptions table, and the notes that have been written up on the patients. Flagging discrepencies for closer examination will help hospitals in the following ways:
1. More accurately track inventory
2. Identify common pain points in data entry
3. Build more robust datasets to work with (less bad data)
4. Treat patients more effectively (more accurate information on treatment a patient has already received)
5. Identify potential fraud

## Setup and Data Exploration
Create the database connection and identify common prescriptions so that we can work with a subset of the data and determine the best model.

In [1]:
import pandas as pd
import mysql.connector
import yaml
import re
from collections import defaultdict
import spacy
import medspacy
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from gpt4all import GPT4All
from pathlib import Path
import spacy
from spacy.pipeline import EntityRuler
import re
import os
from typing import List, Tuple, Any, Optional
import ast

In [2]:
# load config file and connect to MySQL
with open("config.yaml", 'r') as f:
    config = yaml.safe_load(f)

conn = mysql.connector.connect(
    host=config["mysql"]["host"],
    user=config["mysql"]["user"],
    password=config["mysql"]["password"],
    port=config["mysql"].get("port", 3306)
)
conn.database = "healthcare_db"

In [3]:
def execute_query(query):
    try:    
        conn.consume_results()
    except:
        pass
    
    cursor = conn.cursor(buffered=True)
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        columns = [i[0] for i in cursor.description]
        df = pd.DataFrame(result, columns=columns)
    finally:
        cursor.close()
    
    return df

In [4]:
query = """
SELECT
  LOWER(COALESCE(DRUG_NAME_GENERIC, DRUG)) AS drug_name,
  COUNT(*) AS prescription_count,
  COUNT(DISTINCT SUBJECT_ID) AS patient_count
FROM prescriptions
GROUP BY drug_name
ORDER BY prescription_count DESC
LIMIT 10;
"""

prescription_count_df = execute_query(query)
prescription_count_df

,drug_name,prescription_count,patient_count
0,d5w,32823,4021
1,ns,28535,4001
2,potassium chloride,27652,4324
3,furosemide,21260,2869
4,metoprolol,19373,3038
5,magnesium sulfate,12847,3691
6,iso-osmotic dextrose,12133,2986
7,heparin sodium,12125,3505
8,sw,9750,2912
9,sodium chloride 0.9% flush,8927,3154


We will focus on these top drugs today. We will also need to map different names for these drugs so that we will be able to deal with hospital employees using different names for these drugs.

In [5]:
# query to see where raw != generic but generic is not null. Pairs should be unique.
query = """
SELECT
    LOWER(DRUG) AS drug_name,
    LOWER(COALESCE(DRUG_NAME_GENERIC, DRUG)) AS generic_name,
    DRUG AS raw_name
FROM prescriptions
GROUP BY drug_name, generic_name, raw_name
ORDER BY drug_name;
"""

drug_name_mapping_df = execute_query(query)

In [6]:
# find the top generic name with the most raw names
top_generic_name = drug_name_mapping_df.groupby('generic_name').size().idxmax()
top_raw_names = drug_name_mapping_df[drug_name_mapping_df['generic_name'] == top_generic_name]['raw_name'].tolist()
print(f"Top generic name: {top_generic_name}")
print(f"Raw names for {top_generic_name}: {', '.join(top_raw_names)}")

Top generic name: heparin flush
Raw names for heparin flush: HEPARIN, Heparin , Heparin CRRT, Heparin Flush, Heparin Flush (10 units/ml), Heparin Flush (100 units/ml), Heparin Flush CVL  (100 units/ml), Heparin Flush CVL  (100 units/ml) , Heparin Flush Hickman (100 units/ml), Heparin Flush Midline (100 units/ml), Heparin Flush PICC (100 units/ml)


In [7]:
# 1) No two rows share the same raw_name
print(drug_name_mapping_df['raw_name'].nunique() == len(drug_name_mapping_df))

# 2) No missing values in either column
print(drug_name_mapping_df['raw_name'].notna().all() and drug_name_mapping_df['generic_name'].notna().all())

# 3) Each raw_name maps to exactly one generic_name
print((drug_name_mapping_df.groupby('raw_name')['generic_name']
          .nunique() == 1).all())

False
True
False


In [8]:
counts = drug_name_mapping_df.groupby('raw_name')['generic_name'].nunique()

bad_raws = counts[counts > 1].index
bad_cases = (
    drug_name_mapping_df
      .loc[drug_name_mapping_df['raw_name'].isin(bad_raws)]
      .groupby('raw_name')['generic_name']
      .apply(list)
      .reset_index(name='generic_names')
)

print(bad_cases.head(10))

                  raw_name                                      generic_names
0              *NF* Niacin                   [*nf* niacin, nicotinic acid sr]
1                      *nf  [*nf* cilostazol, *nf* moxifloxacin, *nf* tria...
2         Abacavir Sulfate         [abacavir oral solution, abacavir sulfate]
3                  Acetami            [acetaminophen, acetaminophen (liquid)]
4                 Acetamin            [acetaminophen, acetaminophen (liquid)]
5            Acetaminophen  [acetaminophen, acetaminophen (liquid), acetam...
6           Acetaminophen    [acetaminophen (liquid), acetaminophen (rectal)]
7   Acetaminophen (Liquid)            [acetaminophen, acetaminophen (liquid)]
8  Acetaminophen w/Codeine  [acetaminophen w/codeine, acetaminophen w/code...
9           Acetylcysteine  [acetylcysteine, acetylcysteine  20% (neb), ac...


Ok, these many to many relationships are going to be a problem. Today we will ignore because we are going to focus on a subset of the drugs and ensure we have a model that can extract those and their amounts, but in the future we are going to have an issue that will have to be dealt with in the HRP. Hopefully we can get context clues for some and hopefully notes will largely specify say, which type of acetaminophen was administered, or give context clues that we can make a reasonable guess. If not, maybe it makes sense to just flag these as notation was not specific enough.

In [9]:
# find only the raw_names that map to exactly one generic
many_to_one_raws = (
    drug_name_mapping_df
      .groupby('raw_name')['generic_name']
      .nunique()
      .loc[lambda s: s == 1]
      .index
      .tolist()
)

prescriptions = ", ".join(f"'{r}'" for r in many_to_one_raws)

# 3) Query for the top 10 generics among those raws
query = f"""
SELECT
  LOWER(COALESCE(DRUG_NAME_GENERIC, DRUG)) AS generic_name,
  COUNT(*) AS prescription_count
FROM prescriptions
WHERE LOWER(DRUG) IN ({prescriptions})
GROUP BY generic_name
ORDER BY prescription_count DESC
LIMIT 10;
"""

most_common_generics_df = execute_query(query)
most_common_generics_df

,generic_name,prescription_count
0,d5w,32823
1,ns,28535
2,iso-osmotic dextrose,12133
3,sw,9750
4,sodium chloride 0.9% flush,8927
5,lorazepam,8567
6,vancomycin hcl,6788
7,heparin sodium,5713
8,warfarin,5647
9,levofloxacin,4469


In [10]:
raw_to_generic = dict(
    zip(
        drug_name_mapping_df['raw_name'],
        drug_name_mapping_df['generic_name']
    )
)

# for every raw name in the dictionary, add the lowercase version mapping to the same generic
for raw_name in list(raw_to_generic.keys()):
    generic_name = raw_to_generic[raw_name]
    raw_to_generic[raw_name.lower()] = generic_name
    raw_to_generic[raw_name.upper()] = generic_name

# Quick sanity check: look at the first 10 mappings
for raw, gen in list(raw_to_generic.items())[:10]:
    print(f"{raw} --> {gen}")

  -->  
*IND* Pexelizumab/Placebo --> *ind* pexelizumab/placebo
*nf --> *nf* triazolam
*nf*  Loteprednol 0.2% ophth --> *nf*  loteprednol 0.2% ophth
*NF* Alemtuzumab --> *nf* alemtuzumab
*NF* Allopurinol Sodium --> *nf* allopurinol sodium
*NF* Arginine HCl --> *nf* arginine hcl
*NF* Basiliximab --> *nf* basiliximab
*NF* Beclomethasone Dipropionate Inhalation --> *nf* beclomethasone dipropionate inhalation
*NF* Benzoyl Peroxide 5% Wash --> *nf* benzoyl peroxide 5% wash


In [11]:
# query unique categories
query = """
SELECT DISTINCT category
FROM noteevents
WHERE category IS NOT NULL AND category != '';
"""

execute_query(query)

,category
0,Discharge summary
1,Echo
2,ECG
3,Nursing
4,Physician
5,Rehab Services
6,Case Management
7,Respiratory
8,Nutrition
9,General


Now we take 8 of our top 10 generics and query the notes that contain any of the raws, or the generic itself, of that generic. Why only 8? sw and ns will show up as part of a lot of words and be hard to search for. We will do one for the categories nursing, general, discharge, physician, pharmacy, and case management to quickly decide a category to focus on.

In [12]:
def build_query(or_squence, category):
  return f"""
    SELECT
      SUBJECT_ID,
      HADM_ID,
      CHARTDATE,
      CATEGORY,
      TEXT
    FROM NOTEEVENTS
    WHERE LOWER(TEXT) REGEXP '{or_squence}'
    AND CATEGORY = '{category}'
    LIMIT 1;
    """

# drop sw and ns from the top generics
most_common_generics_df = most_common_generics_df[
    ~most_common_generics_df['generic_name'].isin(['sw', 'ns'])
]
top_gens = most_common_generics_df['generic_name'].tolist()

search_terms = {
    raw
    for raw, gen in raw_to_generic.items()
    if gen in top_gens
} | set(top_gens)

or_squence = "|".join(re.escape(term) for term in search_terms)

notes_df = execute_query(build_query(or_squence, 'discharge summary'))
print(f"Found {len(notes_df)} matching notes.")

Found 1 matching notes.


In [13]:
def find_top10_drugs(text):
    text_lower = text.lower()
    found = {
        raw_to_generic.get(term, term)
        for term in search_terms
        if term in text_lower
    }
    return list(found)

def print_first_note(notes_df):
    if not notes_df.empty:
        first_note = notes_df.iloc[0]
        print(f"Top 10 drugs in note: {first_note['top10_drugs_in_note']}")
        print(f"First note text: {first_note['TEXT']}")
    else:
        print("No notes found.")

notes_df['top10_drugs_in_note'] = notes_df['TEXT'].apply(find_top10_drugs)

print_first_note(notes_df)

Top 10 drugs in note: ['heparin sodium', 'levofloxacin', 'lorazepam']
First note text: Admission Date:  [**2118-6-2**]       Discharge Date:  [**2118-6-14**]

Date of Birth:                    Sex:  F

Service:  MICU and then to [**Doctor Last Name **] Medicine

HISTORY OF PRESENT ILLNESS:  This is an 81-year-old female
with a history of emphysema (not on home O2), who presents
with three days of shortness of breath thought by her primary
care doctor to be a COPD flare.  Two days prior to admission,
she was started on a prednisone taper and one day prior to
admission she required oxygen at home in order to maintain
oxygen saturation greater than 90%.  She has also been on
levofloxacin and nebulizers, and was not getting better, and
presented to the [**Hospital1 18**] Emergency Room.

In the [**Hospital3 **] Emergency Room, her oxygen saturation was
100% on CPAP.  She was not able to be weaned off of this
despite nebulizer treatment and Solu-Medrol 125 mg IV x2.

Review of systems is ne

In [14]:
notes_df = execute_query(build_query(or_squence, 'Nursing/other'))
notes_df['top10_drugs_in_note'] = notes_df['TEXT'].apply(find_top10_drugs)
print_first_note(notes_df)

Top 10 drugs in note: ['levofloxacin', 'd5w']
First note text: CSRU NPN

Neuro:  Propofol weaned to 10 mcg/kg/min.  Pt following commands, opens eyes minimally to request.  Denies pain by head nods.  Aggitated/restless at times-versed given w/ effect.  Left pupil 3mm, sluggish but reactive.

CV:  Attempted v pacing for HR initially in 60's nsr->BP down, no change in CI. HR up to 70's NSR with wakefulness.  APC's noted this afternoon.  Lytes repleted as needed.  Neo weaned to off w/ stable BP. Vasopressin continues. Milrinone at .125 mcg/kg/min.  MVO@ mainly in low 60's.  CI 1.8-2 via CCO, 2.75 by FICK. [** **] gtt increased to 800u/hr for PTT 61.8-repeat PTT pending.  Hct stable at 27.

Resp:  Weaned to CPAP w/ 12 IPS to maintain VT's approx 400cc, RR high teens at rest, up to 30's w/ aggitation.  CPAP ABG stable.  Suctioned for small amts clear to yellow sputum.

GI:  Abd soft, ND, hypoactive BS.  TF off x 4 hours for residuals of 130-170cc.  Reglan continues. Restarted criticare at 2

In [15]:
notes_df = execute_query(build_query(or_squence, 'Physician'))
notes_df['top10_drugs_in_note'] = notes_df['TEXT'].apply(find_top10_drugs)

print_first_note(notes_df)

No notes found.


In [16]:
notes_df = execute_query(build_query(or_squence, 'General'))
notes_df['top10_drugs_in_note'] = notes_df['TEXT'].apply(find_top10_drugs)

print_first_note(notes_df)

Top 10 drugs in note: ['heparin sodium']
First note text: Attending MICU Note
   Chief Complaint:  abd pain and  hematuria ,  transient hypotension in
   ED
   I saw and examined the patient, and was physically present with the ICU
   Resident for key portions of the services provided.  I agree with his /
   her note above, including assessment and plan.
   HPI:
   85 yo F with stable CLL, not requiring treatment, presents to ED with
   abd pain, hematuria.
   C/o several month h/o ns, fatigue, poor appetite and intermittent L abd
   discomfort Also notes tinnitus, ongoing new since [**Month (only) **].
   Was seen [**1-9**] by pcp for viral uri sx/dry cough
   Returned to PCP [**1-23**] w/ ongoing sputum production and also had noted to
   have  episode of hematuria and ongoing L LQ pelvic/abd pain.  Treated
   with azithro for presumed CAP/bronchitis.  Labs showed increase in WBC
   to 40 from baseline [**9-24**]'s and blood in urine.  Abd  u/s performed as
   outpt showed splenomega

In [17]:
notes_df = execute_query(build_query(or_squence, 'Pharmacy'))
notes_df['top10_drugs_in_note'] = notes_df['TEXT'].apply(find_top10_drugs)

print_first_note(notes_df)

Top 10 drugs in note: ['vancomycin hcl']
First note text: PHARMACY - VANCOMYCIN
   ASSESSMENT:
   Mr. [**Known lastname 86**] continues on vancomycin 1000 mg q48h (day 11); Currently
   on peritoneal dialysis with continually rising creatinine (2.3 to 5
   mg/dL over 5 to 7 days) and decreasing residual urine output.  Last
   vancomycin dose given [**2-2**] at 8 am; Most recent vanco trough 28.8 @
   8:31 am [**2133-2-4**].
   RECOMMENDATION:
          Hold vancomycin dose today and consider decreasing
   dose/frequency to 500 mg q48h
          Start new regimen [**2-5**] or [**2-6**] if decide to continue
   vancomycin therapy
          Goal vancomycin level 15
 20 mcg/mL
   [**Initials (NamePattern4) **] [**Last Name (NamePattern4) 79**], PharmD #[**Numeric Identifier 80**]



In [18]:
notes_df = execute_query(build_query(or_squence, 'Case Management'))
notes_df['top10_drugs_in_note'] = notes_df['TEXT'].apply(find_top10_drugs)

print_first_note(notes_df)

No notes found.


Using pharmacy notes, they are the clearest and look at a single drug.

In [19]:
query = f"""
SELECT
    TEXT
FROM NOTEEVENTS
WHERE LOWER(TEXT) REGEXP '{or_squence}'
AND CATEGORY = 'Pharmacy'
LIMIT 1000;
"""

notes_df = execute_query(query)
print(f"Found {len(notes_df)} pharmacy notes matching the search terms.")

notes_df['top10_drugs_in_note'] = notes_df['TEXT'].apply(find_top10_drugs)

Found 26 pharmacy notes matching the search terms.


## Extraction:
### Regex
First, were gonna do some standard scraping. This is unlikely to work well, but it will provide a baseline that we will aim to beat.

In [20]:
# build generic to set of aliases map
generic_aliases = defaultdict(set)
for raw, gen in raw_to_generic.items():
    generic_aliases[gen].add(raw)
for gen in list(generic_aliases):
    generic_aliases[gen].add(gen)

# common dosage patterns regex
unit_pattern   = r'(?:mg|g|mcg|μg|units|puffs)'
number_pattern = r'(\d+(?:\.\d+)?)'

def extract_with_aliases(text, drug_list):
    text_lower = text.lower()
    extractions = []
    for gen in drug_list:
        for alias in generic_aliases[gen]:
            esc = re.escape(alias)
            p1 = re.compile(fr'{esc}.{{0,20}}?{number_pattern}\s*{unit_pattern}', re.IGNORECASE)
            p2 = re.compile(fr'{number_pattern}\s*{unit_pattern}.{{0,20}}?{esc}', re.IGNORECASE)
            for pat in (p1, p2):
                for m in pat.finditer(text_lower):
                    extractions.append({
                        m.group(0).strip()
                    })
    return extractions

notes_df['dosage_extractions'] = notes_df.apply(
    lambda row: extract_with_aliases(row['TEXT'], row['top10_drugs_in_note']),
    axis=1
)

In [21]:
print("Note 0")
print(notes_df.iloc[0]['top10_drugs_in_note'])
print(f"\nExtractions: {notes_df.iloc[0]['dosage_extractions']}\n")
print(notes_df.iloc[0]['TEXT'])

Note 0
['vancomycin hcl']

Extractions: [{'vancomycin 1000 mg'}, {'vancomycin 1000 mg'}, {'vancomycin 1000 mg'}]

PHARMACY - VANCOMYCIN
   ASSESSMENT:
   Mr. [**Known lastname 86**] continues on vancomycin 1000 mg q48h (day 11); Currently
   on peritoneal dialysis with continually rising creatinine (2.3 to 5
   mg/dL over 5 to 7 days) and decreasing residual urine output.  Last
   vancomycin dose given [**2-2**] at 8 am; Most recent vanco trough 28.8 @
   8:31 am [**2133-2-4**].
   RECOMMENDATION:
          Hold vancomycin dose today and consider decreasing
   dose/frequency to 500 mg q48h
          Start new regimen [**2-5**] or [**2-6**] if decide to continue
   vancomycin therapy
          Goal vancomycin level 15
 20 mcg/mL
   [**Initials (NamePattern4) **] [**Last Name (NamePattern4) 79**], PharmD #[**Numeric Identifier 80**]



In [22]:
print("Note 1")
print(notes_df.iloc[1]['top10_drugs_in_note'])
print(f"\nExtractions: {notes_df.iloc[1]['dosage_extractions']}\n")
print(notes_df.iloc[1]['TEXT'])

Note 1
['lorazepam']

Extractions: [{'lorazepam 2 mg'}, {'lorazepam 1 mg'}, {'lorazepam 2mg'}, {'lorazepam 1mg'}, {'lorazepam 2 mg'}, {'lorazepam 1 mg'}, {'lorazepam 2mg'}, {'lorazepam 1mg'}, {'lorazepam 2 mg'}, {'lorazepam 1 mg'}, {'lorazepam 2mg'}, {'lorazepam 1mg'}]

Pharmacy Note
   TRANSITIONING and WEANING OPIOIDS: Continue fentanyl infusion and
   initiate methadone intermittent doses (overlap therapy initially).
   Begin to wean fentanyl infusion approximately 3 hours after beginning
   methadone by decreasing the fentanyl infusion by 50% initially.
   MONITORING: Monitor level of analgesia and response to drug as per
   [**Hospital1 54**] sedation/analgesia guideline.
   When fentanyl, propofol and midazolam are weaned off can wean methadone
   as tolerated
 Dose of methadone should be 20mg q6h >> q8h >>
   q12h>>daily.
   Transitioning from midazolam to lorazepam during the 24 hour period in
   anticipation of extubation. Discontinue midazolam. Begin lorazepam 2 mg
   FT q4hr

In [23]:
print("Note 2")
print(notes_df.iloc[2]['top10_drugs_in_note'])
print(f"\nExtractions: {notes_df.iloc[2]['dosage_extractions']}\n")
print(notes_df.iloc[2]['TEXT'])

Note 2
['vancomycin hcl']

Extractions: [{'vancomycin 1 g'}, {'vancomycin 1 g'}, {'vancomycin 1 g'}]

PHARMACY
 VANCO DOSING IN CRRT
   ASSESSMENT - Mr [**Known lastname 86**] is currently on vancomycin 1 gram q48h and is
   now on continuous renal replacement therapy (CRRT) which provides
   variable removal of drugs, but is more efficient than hemodialysis or
   peritoneal dialysis.
   RECOMMENDATION: While on CRRT, drugs may be dosed for an
estimated
   creatinine clearance 20 to 30 mL/minute; mostly dependent on the actual
   ultrafiltration rate.  When levels can be monitored, obtain levels q24h
   and redose drugs to goal (e.g. vancomycin).
     * Vancomycin dose/regimen should likely be changed to 1 gram PRN for
       goal level 15
 20 mcg/mL particularly in the setting of critical
       illness.
     * When vanco level < 20 mcg/mL give 1 gram
     * When vanco level > 20 mcg/mL hold dose and obtain another level 24
       hours later and reassess
   Spoke to Dr. [**Last Name 

I'm actually pretty impressed with how well this has worked, but it is not very robust for several reasons:
1. Using a window to extract a number will fail in cases where language is more complex and we are counting on getting lucky with the regex, our code may pick up the wrong number or no number at all
2. We are able to get units, but something like 1000mg is very different from 1000mg hourly for 24 hours. For this to be useful for our goal, we need to be able to get these aggregations
3. We have multiple differing extractions for certain entites, with no way to differentiate between them, crippling our ability to actually develop an automated system

### Medspacy
We need to improve upon this regex model of scraping. So lets use some nlp tools. I will be using medspacy to, using NER, to hopefully better understand this data and get more comprehensive extraction.

In [24]:
nlp = spacy.load("en_core_web_sm")
ruler = nlp.add_pipe(
    "entity_ruler",
    before="ner",
    config={"phrase_matcher_attr": "LOWER"}
)

patterns = [{"label": "DRUG", "pattern": raw} for raw in raw_to_generic]
ruler.add_patterns(patterns)

unit_pat = r'(?:mg|g|mcg|μg|units|puffs)'
num_pat  = r'(\d+(?:\.\d+)?)'
dosage_re = re.compile(fr'{num_pat}\s*{unit_pat}', re.IGNORECASE)
freq_re   = re.compile(
    r'\b(?:hourly|daily|bid|tid|qid|q\d+h|every\s+\d+\s+(?:hours?|days?))\b',
    re.IGNORECASE
)

In [25]:
def extract_spacy(text):
    doc = nlp(text)
    meds = []
    for ent in doc.ents:
        if ent.label_ != "DRUG":
            continue
        span = ent.text
        generic = raw_to_generic.get(span.lower(), span.lower())
        if generic not in most_common_generics_df['generic_name'].tolist():
            continue
        
        # still need a sliding window around the span for regex
        start = max(ent.start_char - 50, 0)
        end   = min(ent.end_char + 50, len(text))
        window = text[start:end]
        
        dose_m = dosage_re.search(window)
        if not dose_m:
            continue
        
        freq_m = freq_re.search(window)
        meds.append({
            "generic":   generic,
            "matched":   span,
            "strength":  dose_m.group(0),
            "frequency": freq_m.group(0) if freq_m else None
        })
    return meds

In [26]:
notes_df['spacy_meds'] = notes_df['TEXT'].apply(extract_spacy)

num = notes_df['spacy_meds'].apply(bool).sum()
print(f"Extracted meds from {num}/{len(notes_df)} notes")
for _, row in notes_df.head(5).iterrows():
    print(row['spacy_meds'])

Extracted meds from 11/26 notes
[]
[{'generic': 'lorazepam', 'matched': 'lorazepam', 'strength': '2 mg', 'frequency': None}, {'generic': 'lorazepam', 'matched': 'lorazepam', 'strength': '2 mg', 'frequency': None}, {'generic': 'lorazepam', 'matched': 'lorazepam', 'strength': '2mg', 'frequency': 'q24h'}, {'generic': 'lorazepam', 'matched': 'lorazepam', 'strength': '2mg', 'frequency': 'q24h'}]
[]
[{'generic': 'lorazepam', 'matched': 'lorazepam', 'strength': '20 mg', 'frequency': None}, {'generic': 'lorazepam', 'matched': 'lorazepam', 'strength': '2 mg', 'frequency': None}]
[]


I'm dissapointed with how poorly this has worked. Compared to the last version, its outputs are a little better when they occur, but there are less attempts at an answer. Compared to our plain regex, we still have the sliding window and regex reliance which is not good, we still have the issue with repeat, differing drug outputs. The only improvement is that we sometimes have frequencies. This is not worth taking over the plain regex solution as is.

### Llama
The final model I will try will be a Llama model downloaded using gpt4all. This will have hardware performance issues compared to the last 2, but it will have far better memory and understanding across a text and should be able to far better label the drugs. We will create a reusable prompt and feed it in with the texts.

In [27]:
# model_dir = Path(r"D:\gpt4all\models")  
# model_dir.mkdir(exist_ok=True)

# model = GPT4All(
#     model_name="Meta-Llama-3-8B-Instruct.Q4_0.gguf",
#     model_path=model_dir,
#     allow_download=True,
#     n_threads=4
# )

In [28]:
def find_raws_in_text(generics, text):
    text_lower = text.lower()
    raws = []
    for gen in generics:
        for raw in generic_aliases.get(gen, []):
            if raw in text_lower:
                raws.append(raw)
    return raws

In [29]:
notes_df['raws_in_note'] = notes_df.apply(
    lambda row: find_raws_in_text(row['top10_drugs_in_note'], row['TEXT']),
    axis=1
)
notes_df.head(1)

,TEXT,top10_drugs_in_note,dosage_extractions,spacy_meds,raws_in_note
0,PHARMACY - VANCOMYCIN\n ASSESSMENT:\n Mr. ...,[vancomycin hcl],"[{vancomycin 1000 mg}, {vancomycin 1000 mg}, {...",[],[vancomycin ]


In [ ]:
# def make_prompt(target_words, text):
#     prompt = (
#         "You are a medical expert specializing in drug dosage extraction. "
#         "Your task is to identify and extract the dosage information for the following drugs: "
#         f"{', '.join(target_words)}.\n\n"
#         "For each drug, provide the dosage in the format:\n"
#         "drug_name,dosage_value,unit,frequency\n"
#         "If a field is not present in the text, use 'None' for that field.\n\n"
#         "Here is the text to analyze:\n"
#         f"{text}\n\n"
#         "Return one line per drug."
#     )
#     return prompt

# def extract_with_gpt4all(row, max_tokens=256):
#     drugs = row["raws_in_note"]
#     if not drugs:
#         return ""
#     prompt = make_prompt(drugs, row["TEXT"])

#     response = model.generate(prompt, n_predict=max_tokens)
#     return response.strip()

#### No Memory, no gpu, shit computer
I was not able to run this on my machine, so in Supplement_LLM.ipynb which is also attached with this assignment you can see how I got the prompted outputs from Llama. I ran supplement_LLM with a GPU in colab and copy and pasted the outputs below.

In [31]:
# From GPU enabled environment: Supplement_LLM.ipynb
output1 = "(vancomycin,1000 mg,q48h)"
output2 = "(methadone,20mg,q6h), (fentanyl,350mcg/hr,), (midazolam,10mg/hr,), (propofol,75mcg/kg/min,), (lorazepam,2mg,q4hr),(lorazepam,1mg,q4prn)"
output3 = "(vancomycin,1 gram,q48h), (vancomycin,1 gram,PRN),(vancomycin,1 gram,when level <20 mcg/mL),(vancomycin,hold dose when level >20 mcg/mL)"

In [32]:
note = notes_df.iloc[0]
print(f"Drugs in note: {note['raws_in_note']}")
print(output1)
print(f"text: {note['TEXT']}")

Drugs in note: ['vancomycin ']
(vancomycin,1000 mg,q48h)
text: PHARMACY - VANCOMYCIN
   ASSESSMENT:
   Mr. [**Known lastname 86**] continues on vancomycin 1000 mg q48h (day 11); Currently
   on peritoneal dialysis with continually rising creatinine (2.3 to 5
   mg/dL over 5 to 7 days) and decreasing residual urine output.  Last
   vancomycin dose given [**2-2**] at 8 am; Most recent vanco trough 28.8 @
   8:31 am [**2133-2-4**].
   RECOMMENDATION:
          Hold vancomycin dose today and consider decreasing
   dose/frequency to 500 mg q48h
          Start new regimen [**2-5**] or [**2-6**] if decide to continue
   vancomycin therapy
          Goal vancomycin level 15
 20 mcg/mL
   [**Initials (NamePattern4) **] [**Last Name (NamePattern4) 79**], PharmD #[**Numeric Identifier 80**]



In [33]:
note = notes_df.iloc[1]
print(f"Drugs in note: {note['raws_in_note']}")
print(output2)
print(f"text: {note['TEXT']}")

Drugs in note: ['lorazepam']
(methadone,20mg,q6h), (fentanyl,350mcg/hr,), (midazolam,10mg/hr,), (propofol,75mcg/kg/min,), (lorazepam,2mg,q4hr),(lorazepam,1mg,q4prn)
text: Pharmacy Note
   TRANSITIONING and WEANING OPIOIDS: Continue fentanyl infusion and
   initiate methadone intermittent doses (overlap therapy initially).
   Begin to wean fentanyl infusion approximately 3 hours after beginning
   methadone by decreasing the fentanyl infusion by 50% initially.
   MONITORING: Monitor level of analgesia and response to drug as per
   [**Hospital1 54**] sedation/analgesia guideline.
   When fentanyl, propofol and midazolam are weaned off can wean methadone
   as tolerated
 Dose of methadone should be 20mg q6h >> q8h >>
   q12h>>daily.
   Transitioning from midazolam to lorazepam during the 24 hour period in
   anticipation of extubation. Discontinue midazolam. Begin lorazepam 2 mg
   FT q4hr and lorazepam 1 mg IV q4hr:prn
   [**2142-2-22**]  1400
   Begin methadone 20mg IV q6hrs
   Fentany

In [34]:
note = notes_df.iloc[2]
print(f"Drugs in note: {note['raws_in_note']}")
print(output2)
print(f"text: {note['TEXT']}")

Drugs in note: ['vancomycin ']
(methadone,20mg,q6h), (fentanyl,350mcg/hr,), (midazolam,10mg/hr,), (propofol,75mcg/kg/min,), (lorazepam,2mg,q4hr),(lorazepam,1mg,q4prn)
text: PHARMACY
 VANCO DOSING IN CRRT
   ASSESSMENT - Mr [**Known lastname 86**] is currently on vancomycin 1 gram q48h and is
   now on continuous renal replacement therapy (CRRT) which provides
   variable removal of drugs, but is more efficient than hemodialysis or
   peritoneal dialysis.
   RECOMMENDATION: While on CRRT, drugs may be dosed for an
estimated
   creatinine clearance 20 to 30 mL/minute; mostly dependent on the actual
   ultrafiltration rate.  When levels can be monitored, obtain levels q24h
   and redose drugs to goal (e.g. vancomycin).
     * Vancomycin dose/regimen should likely be changed to 1 gram PRN for
       goal level 15
 20 mcg/mL particularly in the setting of critical
       illness.
     * When vanco level < 20 mcg/mL give 1 gram
     * When vanco level > 20 mcg/mL hold dose and obtain another

This is by far the best of the options. It is smart enough to not output a bunch of copies of the same drug unless there is different for frequency or dosage and does a great job, to my eyes, of picking everything up accuracy. Also, there is no reliance on regex and sliding windows, which makes this method far more robust and transferrable to different types of documents and new drugs. Interestingly, the second option added correct information for drugs that weren't even in our list, which does tell me that I need to prompt more carefully, but also shows the model's potential to be generalized to my larger project.

## Citation

@misc{gpt4all,
  author = {Yuvanesh Anand and Zach Nussbaum and Brandon Duderstadt and Benjamin Schmidt and Andriy Mulyar},
  title = {GPT4All: Training an Assistant-style Chatbot with Large Scale Data Distillation from GPT-3.5-Turbo},
  year = {2023},
  publisher = {GitHub},
  journal = {GitHub repository},
  howpublished = {\url{https://github.com/nomic-ai/gpt4all}},
}

## Flagging
Now that we know what the best solution is for this extraction, we need to use this solution on all pharmacy notes with corresponding prescription entries so that we can make comparisons and begin flagging. For the real thing, I will take 1000 pharmacy noteevents and their corresponding prescription events, feed the notes into the pharmacy table, and compare extracted values from notes to prescription table.

In [35]:
query_all_pharmacy_notes_and_perscriptions = """
WITH pharmacy_notes AS (
  SELECT *
  FROM NOTEEVENTS
  WHERE CATEGORY = 'Pharmacy'
  AND TEXT IS NOT NULL
  LIMIT 1000
)

SELECT
  n.ROW_ID AS note_row_id,
  n.SUBJECT_ID,
  n.HADM_ID,
  n.CHARTDATE,
  n.TEXT AS note_text,
  p.ROW_ID AS prescription_row_id,
  p.DRUG,
  p.DRUG_NAME_GENERIC,
  p.DRUG_TYPE,
  p.PROD_STRENGTH,
  p.DOSE_VAL_RX,
  p.DOSE_UNIT_RX,
  p.STARTDATE,
  p.ENDDATE
FROM pharmacy_notes n
LEFT JOIN PRESCRIPTIONS p
  ON n.SUBJECT_ID = p.SUBJECT_ID
  AND n.HADM_ID = p.HADM_ID
ORDER BY n.ROW_ID;"""

full_table = execute_query(query_all_pharmacy_notes_and_perscriptions)
full_table.head(5)

,note_row_id,SUBJECT_ID,HADM_ID,CHARTDATE,note_text,prescription_row_id,DRUG,DRUG_NAME_GENERIC,DRUG_TYPE,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,STARTDATE,ENDDATE
0,314570,30354,120396,2154-12-25,TITLE: PHARMACY\n SEDATION\n Mr. [**Known la...,None,None,None,None,None,None,None,None,None
1,314572,30354,120396,2154-12-25,TITLE: PHARMACY\n SEDATION\n ASSESSMENT:\n ...,None,None,None,None,None,None,None,None,None
2,314582,27063,139787,2133-02-04,PHARMACY - VANCOMYCIN\n ASSESSMENT:\n Mr. ...,None,None,None,None,None,None,None,None,None
3,314707,27063,139787,2133-02-05,PHARMACY\n VANCO DOSING IN CRRT\n ASSESSMENT...,None,None,None,None,None,None,None,None,None
4,314739,29200,105865,2142-02-22,Pharmacy Note\n TRANSITIONING and WEANING OP...,None,None,None,None,None,None,None,None,None


In [36]:
full_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445 entries, 0 to 444
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   note_row_id          445 non-null    object
 1   SUBJECT_ID           445 non-null    object
 2   HADM_ID              444 non-null    object
 3   CHARTDATE            445 non-null    object
 4   note_text            445 non-null    object
 5   prescription_row_id  345 non-null    object
 6   DRUG                 345 non-null    object
 7   DRUG_NAME_GENERIC    217 non-null    object
 8   DRUG_TYPE            345 non-null    object
 9   PROD_STRENGTH        345 non-null    object
 10  DOSE_VAL_RX          345 non-null    object
 11  DOSE_UNIT_RX         345 non-null    object
 12  STARTDATE            345 non-null    object
 13  ENDDATE              345 non-null    object
dtypes: object(14)
memory usage: 48.8+ KB


In [37]:
# save to csv
df_unique = full_table.drop_duplicates(subset='note_row_id')[['note_row_id', 'note_text']].copy()
df_unique['note_text'] = df_unique['note_text'].str.replace('"', "'", regex=False)
df_unique['note_text'] = '"' + df_unique['note_text'] + '"'
df_unique.to_csv('unique_pharmacy_notes.csv', index=False)

Now read our extractions and get lists of tuples entered into a dict

In [38]:
def clean_token(token):
    if token is None:
        return None
    token = str(token).strip().strip('"').strip("'")
    if token == "" or token.lower() == "none":
        return None
    return token

def extract_tuples_fuzzy(text: str):
    """
    Extract tuples like (drug, dose, freq, ...) from free-form LLM text.
    Works even if strings aren't quoted and multiple blocks exist.
    """
    if not isinstance(text, str):
        return []
    
    tuples_raw = re.findall(r"\(([^()]+)\)", text, flags=re.MULTILINE)

    results = []
    for t in tuples_raw:
        parts = [p.strip() for p in t.split(",")]
        if len(parts) > 4:
            parts = parts[:3] + [",".join(parts[3:])]
        while len(parts) < 4:
            parts.append(None)

        parts = [clean_token(p) for p in parts]
        if any(p is not None for p in parts):
            results.append(tuple(parts))

    return results

In [39]:
df = pd.read_csv("extracted_pharmacy_notes.csv")

row_to_extraction = {}
for _, row in df.iterrows():
    note_id = row["note_row_id"]
    ext_text = row.get("extraction", "")
    parsed = extract_tuples_fuzzy(ext_text)
    row_to_extraction[note_id] = parsed

df["extraction_parsed"] = df["extraction"].apply(extract_tuples_fuzzy)
print(row_to_extraction)

{314570: [('Fentanyl', '50 mcg/hr', 'hourly', None), ('Midazolam', '2 mg/hr', 'hourly', None)], 314572: [('fentanyl', '25 mcg/hr', None, None), ('midazolam', '1 mg/hr', None, None), ('versed', '12 mg', None, ',')], 314582: [('vancomycin', '1000 mg', 'q48h', None), ('vancomycin', '1000 mg', 'q48h', None)], 314707: [('VANCOMYCIN', '1 gram', 'PRN', '-'), ('VANCOMYCIN', '1 gram', None, None), ('VANCOMYCIN', '-', 'mcg/mL', '<20'), ('VANCOMYCIN', '-', 'mcg/mL', '>20'), ('VANCOMYCIN', '-', 'mcg/mL', None), ('micrograms per milliliter', None, None, None)], 314739: [('methadone', '20mg', 'q6h', None), ('fentanyl', '350mcg/hr', None, None), ('midazolam', '10mg/hr', None, None), ('propofol', '75mcg/kg/min', None, None), ('methadone', '20mg', 'q6h', None), ('fentanyl', '175mcg/hr', None, None), ('midazolam', '10mg/hr', None, None), ('propofol', '75mcg/kg/min', None, None), ('methadone', '20mg', 'q6h', None), ('fentanyl', '0mcg/hr', None, None), ('midazolam', '10mg/hr', None, None), ('propofol', '3

Now we have our extractions, lets flag rows with missing data

In [42]:
_MASS_UNITS = {
    "mcg": "mcg", "μg": "mcg", "ug": "mcg", "microgram": "mcg", "micrograms": "mcg",
    "mg": "mg", "milligram": "mg", "milligrams": "mg",
    "g": "g", "gm": "g", "gram": "g", "grams": "g"
}

def parse_mass(text):
    """Extract (value_float, unit_canonical) from a string like '50 mcg/hr'.
       Ignores frequency/rate. Returns (None, None) if not found."""
    if not isinstance(text, str):
        return None, None
    s = text.strip().lower()
    m = re.search(r'(\d+(?:\.\d+)?)', s)
    if not m:
        return None, None
    val = float(m.group(1))
    after = s[m.end():]
    for tok in re.findall(r'[a-z]+', after):
        if tok in _MASS_UNITS:
            return val, _MASS_UNITS[tok]
    before = s[:m.start()]
    for tok in reversed(re.findall(r'[a-z]+', before)):
        if tok in _MASS_UNITS:
            return val, _MASS_UNITS[tok]
    return None, None

DRUG_SYNONYMS = {"versed": "midazolam", "zyprexa": "olanzapine"}

def norm_str(s):
    return str(s).strip().lower() if isinstance(s, str) else None

def canonical_drug(name):
    n = norm_str(name)
    return DRUG_SYNONYMS.get(n, n)

def normalize_text(s: str) -> str:
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", " ", s)
    return f" {s.strip()} "

def drug_matches(drug_can: str, rx_row) -> bool:
    """Exact canonical OR word-boundary substring match in DRUG/DRUG_NAME_GENERIC."""
    if not drug_can:
        return False
    if rx_row.get("drug_norm") == drug_can:
        return True
    if rx_row.get("drug_generic_norm") == drug_can:
        return True
    needle = f" {drug_can} "
    for col in ("DRUG", "DRUG_NAME_GENERIC"):
        txt = rx_row.get(col)
        if isinstance(txt, str) and needle in normalize_text(txt):
            return True
    return False

def doses_equal(val_a, unit_a, val_b, unit_b, tol=1e-6):
    """Require same mass unit + numeric equality (tiny tolerance)."""
    if unit_a is None or unit_b is None:
        return False
    if unit_a != unit_b:
        return False
    if val_a is None or val_b is None:
        return False
    return abs(float(val_a) - float(val_b)) <= tol

def ensure_list(obj):
    if isinstance(obj, list):
        return obj
    try:
        import ast
        return ast.literal_eval(obj)
    except Exception:
        return []

In [43]:
df["extraction_parsed"] = df["extraction_parsed"].apply(ensure_list)
df["note_row_id"] = df["note_row_id"].astype(int)
full_table["note_row_id"] = full_table["note_row_id"].astype(int)

full_table["drug_norm"] = full_table["DRUG"].apply(canonical_drug)
full_table["drug_generic_norm"] = (
    full_table["DRUG_NAME_GENERIC"].apply(canonical_drug)
    if "DRUG_NAME_GENERIC" in full_table.columns else None
)

def unit_from_dose_unit_norm(u):
    """dose_unit_norm may be things like 'mg', 'mcg', or 'mg/hr'. Keep mass unit only."""
    if not isinstance(u, str):
        return None
    _, mass = parse_mass(f"1 {u}")
    return mass

def compute_rx_val_unit_final(row):
    if "rx_val" in row and "rx_unit" in row:
        rv, ru = row["rx_val"], row["rx_unit"]
        if pd.notna(rv) and isinstance(ru, str):
            if ru in _MASS_UNITS:
                return float(rv), ru
            else:
                ru_mass = unit_from_dose_unit_norm(ru)
                if ru_mass:
                    return float(rv), ru_mass

    if "dose_val_norm" in row and "dose_unit_norm" in row:
        dv, du = row["dose_val_norm"], row["dose_unit_norm"]
        if pd.notna(dv) and du is not None:
            du_mass = unit_from_dose_unit_norm(du)
            if du_mass:
                return float(dv), du_mass

    ps = row.get("PROD_STRENGTH")
    vv, uu = parse_mass(ps) if isinstance(ps, str) else (None, None)
    if vv is not None and uu is not None:
        return vv, uu

    vv, uu = parse_mass(row.get("DRUG")) if isinstance(row.get("DRUG"), str) else (None, None)
    if vv is not None and uu is not None:
        return vv, uu

    vv, uu = parse_mass(row.get("DRUG_NAME_GENERIC")) if isinstance(row.get("DRUG_NAME_GENERIC"), str) else (None, None)
    if vv is not None and uu is not None:
        return vv, uu

    return None, None

rx_final = full_table.apply(compute_rx_val_unit_final, axis=1, result_type="reduce")
full_table["rx_val_final"]  = [p[0] for p in rx_final]
full_table["rx_unit_final"] = [p[1] for p in rx_final]

print(
    "Usable Rx dose/units:",
    int(full_table["rx_val"].notna().sum() if "rx_val" in full_table.columns else 0), "(rx_val) ->",
    int(full_table["rx_val_final"].notna().sum()), "(after fallbacks)"
)

rx_by_note = {k: g for k, g in full_table.groupby("note_row_id")}

rows = []
for _, r in df.iterrows():
    nid = int(r["note_row_id"])
    tuples = r.get("extraction_parsed", []) or []
    rxg = rx_by_note.get(nid, pd.DataFrame())

    for idx, t in enumerate(tuples):
        drug = t[0] if len(t) > 0 else None
        dose_raw = t[1] if len(t) > 1 else None

        drug_can = canonical_drug(drug)
        ex_val, ex_unit = parse_mass(dose_raw)

        matched = False
        drug_expected = None
        rx_val_match = None
        rx_unit_match = None

        if drug_can and ex_val is not None and ex_unit is not None and not rxg.empty:
            cand = rxg[rxg.apply(lambda rr: drug_matches(drug_can, rr), axis=1)]
            if not cand.empty:
                for _, rx in cand.iterrows():
                    if doses_equal(ex_val, ex_unit, rx["rx_val_final"], rx["rx_unit_final"]):
                        matched = True
                        drug_expected = (
                            rx["DRUG_NAME_GENERIC"] if isinstance(rx.get("DRUG_NAME_GENERIC"), str)
                            else rx["DRUG"]
                        )
                        rx_val_match = rx["rx_val_final"]
                        rx_unit_match = rx["rx_unit_final"]
                        break

        rows.append({
            "note_row_id": nid,
            "tuple_index": idx,
            "drug_extracted": drug,
            "dose_raw": dose_raw,
            "dose_val_extracted": ex_val,
            "dose_unit_extracted": ex_unit,
            "drug_expected": drug_expected,
            "rx_val_matched": rx_val_match,
            "rx_unit_matched": rx_unit_match,
            "matches_prescription": bool(matched),
        })

results = pd.DataFrame(rows)
matches_df = results[results["matches_prescription"]].reset_index(drop=True)
mismatches_df = results[~results["matches_prescription"]].reset_index(drop=True)

Usable Rx dose/units: 0 (rx_val) -> 249 (after fallbacks)


In [44]:
matches_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   note_row_id           1 non-null      int64  
 1   tuple_index           1 non-null      int64  
 2   drug_extracted        1 non-null      object 
 3   dose_raw              1 non-null      object 
 4   dose_val_extracted    1 non-null      float64
 5   dose_unit_extracted   1 non-null      object 
 6   drug_expected         1 non-null      object 
 7   rx_val_matched        1 non-null      float64
 8   rx_unit_matched       1 non-null      object 
 9   matches_prescription  1 non-null      bool   
dtypes: bool(1), float64(2), int64(2), object(5)
memory usage: 205.0+ bytes


In [45]:
mismatches_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553 entries, 0 to 552
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   note_row_id           553 non-null    int64  
 1   tuple_index           553 non-null    int64  
 2   drug_extracted        553 non-null    object 
 3   dose_raw              418 non-null    object 
 4   dose_val_extracted    309 non-null    float64
 5   dose_unit_extracted   309 non-null    object 
 6   drug_expected         0 non-null      object 
 7   rx_val_matched        0 non-null      float64
 8   rx_unit_matched       0 non-null      object 
 9   matches_prescription  553 non-null    bool   
dtypes: bool(1), float64(2), int64(2), object(5)
memory usage: 39.6+ KB


In [46]:
matches_df.head(10)

,note_row_id,tuple_index,drug_extracted,dose_raw,dose_val_extracted,dose_unit_extracted,drug_expected,rx_val_matched,rx_unit_matched,matches_prescription
0,327518,0,Aliskiren,150-300 mg,150.0,mg,*NF* Aliskiren,150.0,mg,True


In [49]:
mismatches_df.head(10)

,note_row_id,tuple_index,drug_extracted,dose_raw,dose_val_extracted,dose_unit_extracted,drug_expected,rx_val_matched,rx_unit_matched,matches_prescription
0,314570,0,Fentanyl,50 mcg/hr,50.0,mcg,None,NaN,None,False
1,314570,1,Midazolam,2 mg/hr,2.0,mg,None,NaN,None,False
2,314572,0,fentanyl,25 mcg/hr,25.0,mcg,None,NaN,None,False
3,314572,1,midazolam,1 mg/hr,1.0,mg,None,NaN,None,False
4,314572,2,versed,12 mg,12.0,mg,None,NaN,None,False
5,314582,0,vancomycin,1000 mg,1000.0,mg,None,NaN,None,False
6,314582,1,vancomycin,1000 mg,1000.0,mg,None,NaN,None,False
7,314707,0,VANCOMYCIN,1 gram,1.0,g,None,NaN,None,False
8,314707,1,VANCOMYCIN,1 gram,1.0,g,None,NaN,None,False
9,314707,2,VANCOMYCIN,-,NaN,None,None,NaN,None,False


In [50]:
full_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445 entries, 0 to 444
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   note_row_id          445 non-null    int64  
 1   SUBJECT_ID           445 non-null    object 
 2   HADM_ID              444 non-null    object 
 3   CHARTDATE            445 non-null    object 
 4   note_text            445 non-null    object 
 5   prescription_row_id  345 non-null    object 
 6   DRUG                 345 non-null    object 
 7   DRUG_NAME_GENERIC    217 non-null    object 
 8   DRUG_TYPE            345 non-null    object 
 9   PROD_STRENGTH        345 non-null    object 
 10  DOSE_VAL_RX          345 non-null    object 
 11  DOSE_UNIT_RX         345 non-null    object 
 12  STARTDATE            345 non-null    object 
 13  ENDDATE              345 non-null    object 
 14  drug_norm            345 non-null    object 
 15  drug_generic_norm    217 non-null    obj

In [51]:
full_table[full_table['note_row_id'] == 314570]

,note_row_id,SUBJECT_ID,HADM_ID,CHARTDATE,note_text,prescription_row_id,DRUG,DRUG_NAME_GENERIC,DRUG_TYPE,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,STARTDATE,ENDDATE,drug_norm,drug_generic_norm,rx_val_final,rx_unit_final
0,314570,30354,120396,2154-12-25,TITLE: PHARMACY\n SEDATION\n Mr. [**Known la...,None,None,None,None,None,None,None,None,None,None,None,NaN,None


Failing because my mysql data is not complete, going back to the roots with PRESCRIPTIONS.csv

In [52]:
query_pharmacy_notes = """
SELECT
  ROW_ID   AS note_row_id,
  SUBJECT_ID,
  HADM_ID,
  CHARTDATE,
  TEXT     AS note_text
FROM NOTEEVENTS
WHERE CATEGORY = 'Pharmacy'
  AND TEXT IS NOT NULL
ORDER BY ROW_ID
LIMIT 1000;
"""

notes_df = execute_query(query_pharmacy_notes)
print("Notes shape:", notes_df.shape)

def to_key_str(series: pd.Series) -> pd.Series:
    if pd.api.types.is_integer_dtype(series):
        return series.astype("Int64").astype(str)
    return series.astype(str)

notes_df["SUBJECT_ID"] = to_key_str(notes_df["SUBJECT_ID"])
notes_df["HADM_ID"]    = to_key_str(notes_df["HADM_ID"])

notes_pairs = (
    notes_df[["SUBJECT_ID", "HADM_ID"]]
    .dropna()
    .drop_duplicates()
)
pairs_key = set(map(tuple, notes_pairs.values.tolist()))
print("Unique (SUBJECT_ID, HADM_ID) pairs in notes:", len(pairs_key))

usecols = [
    "ROW_ID", "SUBJECT_ID", "HADM_ID",
    "DRUG", "DRUG_NAME_GENERIC", "DRUG_TYPE",
    "PROD_STRENGTH", "DOSE_VAL_RX", "DOSE_UNIT_RX",
    "STARTDATE", "ENDDATE",
]

chunksize = 250_000
kept_chunks = []

reader = pd.read_csv(
    "prescriptions.csv",
    usecols=usecols,
    dtype={"SUBJECT_ID": str, "HADM_ID": str},
    chunksize=chunksize,
    low_memory=True
)

total_rows = 0
kept_rows = 0
for i, chunk in enumerate(reader, start=1):
    total_rows += len(chunk)

    chunk["SUBJECT_ID"] = to_key_str(chunk["SUBJECT_ID"])
    chunk["HADM_ID"]    = to_key_str(chunk["HADM_ID"])

    mask = chunk.apply(lambda r: (r["SUBJECT_ID"], r["HADM_ID"]) in pairs_key, axis=1)
    filtered = chunk[mask]
    kept_rows += len(filtered)

    if not filtered.empty:
        kept_chunks.append(filtered)

    if i % 10 == 0:
        print(f"Processed ~{i * chunksize:,} rows; kept so far: {kept_rows:,}")

rx_df_small = pd.concat(kept_chunks, ignore_index=True) if kept_chunks else pd.DataFrame(columns=usecols)
print("Prescriptions total rows scanned:", f"{total_rows:,}")
print("Prescriptions kept (matching pairs):", rx_df_small.shape)

full_table = notes_df.merge(
    rx_df_small,
    on=["SUBJECT_ID", "HADM_ID"],
    how="left",
    suffixes=("", "_rx")
)

full_table = full_table.rename(columns={"ROW_ID": "prescription_row_id"})

desired_order = [
    "note_row_id", "SUBJECT_ID", "HADM_ID", "CHARTDATE", "note_text",
    "prescription_row_id", "DRUG", "DRUG_NAME_GENERIC", "DRUG_TYPE",
    "PROD_STRENGTH", "DOSE_VAL_RX", "DOSE_UNIT_RX", "STARTDATE", "ENDDATE"
]

cols = [c for c in desired_order if c in full_table.columns] + \
       [c for c in full_table.columns if c not in desired_order]
full_table = full_table[cols]

print("Joined shape:", full_table.shape)
display(full_table.head(10))

Notes shape: (103, 5)
Unique (SUBJECT_ID, HADM_ID) pairs in notes: 69
Processed ~2,500,000 rows; kept so far: 5,939
Prescriptions total rows scanned: 4,156,450
Prescriptions kept (matching pairs): (17139, 11)
Joined shape: (28126, 14)


,note_row_id,SUBJECT_ID,HADM_ID,CHARTDATE,note_text,prescription_row_id,DRUG,DRUG_NAME_GENERIC,DRUG_TYPE,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,STARTDATE,ENDDATE
0,314570,30354,120396,2154-12-25,TITLE: PHARMACY\n SEDATION\n Mr. [**Known la...,100239.0,NS,NaN,BASE,50mL Bag,50,mL,2154-12-26 00:00:00,2154-12-26 00:00:00
1,314570,30354,120396,2154-12-25,TITLE: PHARMACY\n SEDATION\n Mr. [**Known la...,100055.0,Lorazepam,Lorazepam,MAIN,2mg/mL Syringe,0.5,mg,2154-12-26 00:00:00,2154-12-26 00:00:00
2,314570,30354,120396,2154-12-25,TITLE: PHARMACY\n SEDATION\n Mr. [**Known la...,99892.0,Lansoprazole Oral Disintegrating Tab,Lansoprazole Oral Disintegrating Tab,MAIN,30mg SoluTab,30,mg,2154-12-16 00:00:00,2154-12-26 00:00:00
3,314570,30354,120396,2154-12-25,TITLE: PHARMACY\n SEDATION\n Mr. [**Known la...,100027.0,Lactulose,Lactulose,MAIN,20g/30mL Cup,30,mL,2154-12-17 00:00:00,2154-12-19 00:00:00
4,314570,30354,120396,2154-12-25,TITLE: PHARMACY\n SEDATION\n Mr. [**Known la...,100029.0,Insulin,Insulin Glargine,MAIN,10mLVial,8,UNIT,2154-12-17 00:00:00,2154-12-25 00:00:00
5,314570,30354,120396,2154-12-25,TITLE: PHARMACY\n SEDATION\n Mr. [**Known la...,100028.0,Insulin,Insulin - Sliding Scale,MAIN,Dummy Package for Sliding Scale,0,UNIT,2154-12-17 00:00:00,2154-12-25 00:00:00
6,314570,30354,120396,2154-12-25,TITLE: PHARMACY\n SEDATION\n Mr. [**Known la...,99897.0,Bisacodyl,Bisacodyl (Rectal),MAIN,10mg Suppository,10,mg,2154-12-17 00:00:00,2154-12-26 00:00:00
7,314570,30354,120396,2154-12-25,TITLE: PHARMACY\n SEDATION\n Mr. [**Known la...,99896.0,Bisacodyl,Bisacodyl,MAIN,5 mg Tab,10,mg,2154-12-17 00:00:00,2154-12-26 00:00:00
8,314570,30354,120396,2154-12-25,TITLE: PHARMACY\n SEDATION\n Mr. [**Known la...,100032.0,Furosemide,Furosemide,MAIN,40mg/4mL Vial,20,mg,2154-12-19 00:00:00,2154-12-20 00:00:00
9,314570,30354,120396,2154-12-25,TITLE: PHARMACY\n SEDATION\n Mr. [**Known la...,100030.0,Metoclopramide,Metoclopramide,MAIN,10mg Tab,5,mg,2154-12-19 00:00:00,2154-12-26 00:00:00


In [53]:
full_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28126 entries, 0 to 28125
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   note_row_id          28126 non-null  object 
 1   SUBJECT_ID           28126 non-null  object 
 2   HADM_ID              28126 non-null  object 
 3   CHARTDATE            28126 non-null  object 
 4   note_text            28126 non-null  object 
 5   prescription_row_id  28125 non-null  float64
 6   DRUG                 28125 non-null  object 
 7   DRUG_NAME_GENERIC    13492 non-null  object 
 8   DRUG_TYPE            28125 non-null  object 
 9   PROD_STRENGTH        28125 non-null  object 
 10  DOSE_VAL_RX          28125 non-null  object 
 11  DOSE_UNIT_RX         28125 non-null  object 
 12  STARTDATE            28119 non-null  object 
 13  ENDDATE              28119 non-null  object 
dtypes: float64(1), object(13)
memory usage: 3.0+ MB


In [54]:
df["extraction_parsed"] = df["extraction_parsed"].apply(ensure_list)
df["note_row_id"] = df["note_row_id"].astype(int)
full_table["note_row_id"] = full_table["note_row_id"].astype(int)

full_table["drug_norm"] = full_table["DRUG"].apply(canonical_drug)
full_table["drug_generic_norm"] = (
    full_table["DRUG_NAME_GENERIC"].apply(canonical_drug)
    if "DRUG_NAME_GENERIC" in full_table.columns else None
)

def unit_from_dose_unit_norm(u):
    """dose_unit_norm may be things like 'mg', 'mcg', or 'mg/hr'. Keep mass unit only."""
    if not isinstance(u, str):
        return None
    _, mass = parse_mass(f"1 {u}")
    return mass

def compute_rx_val_unit_final(row):
    if "rx_val" in row and "rx_unit" in row:
        rv, ru = row["rx_val"], row["rx_unit"]
        if pd.notna(rv) and isinstance(ru, str):
            if ru in _MASS_UNITS:
                return float(rv), ru
            else:
                ru_mass = unit_from_dose_unit_norm(ru)
                if ru_mass:
                    return float(rv), ru_mass

    if "dose_val_norm" in row and "dose_unit_norm" in row:
        dv, du = row["dose_val_norm"], row["dose_unit_norm"]
        if pd.notna(dv) and du is not None:
            du_mass = unit_from_dose_unit_norm(du)
            if du_mass:
                return float(dv), du_mass

    ps = row.get("PROD_STRENGTH")
    vv, uu = parse_mass(ps) if isinstance(ps, str) else (None, None)
    if vv is not None and uu is not None:
        return vv, uu

    vv, uu = parse_mass(row.get("DRUG")) if isinstance(row.get("DRUG"), str) else (None, None)
    if vv is not None and uu is not None:
        return vv, uu

    vv, uu = parse_mass(row.get("DRUG_NAME_GENERIC")) if isinstance(row.get("DRUG_NAME_GENERIC"), str) else (None, None)
    if vv is not None and uu is not None:
        return vv, uu

    return None, None

rx_final = full_table.apply(compute_rx_val_unit_final, axis=1, result_type="reduce")
full_table["rx_val_final"]  = [p[0] for p in rx_final]
full_table["rx_unit_final"] = [p[1] for p in rx_final]

print(
    "Usable Rx dose/units:",
    int(full_table["rx_val"].notna().sum() if "rx_val" in full_table.columns else 0), "(rx_val) ->",
    int(full_table["rx_val_final"].notna().sum()), "(after fallbacks)"
)

rx_by_note = {k: g for k, g in full_table.groupby("note_row_id")}

rows = []
for _, r in df.iterrows():
    nid = int(r["note_row_id"])
    tuples = r.get("extraction_parsed", []) or []
    rxg = rx_by_note.get(nid, pd.DataFrame())

    for idx, t in enumerate(tuples):
        drug = t[0] if len(t) > 0 else None
        dose_raw = t[1] if len(t) > 1 else None

        drug_can = canonical_drug(drug)
        ex_val, ex_unit = parse_mass(dose_raw)

        matched = False
        drug_expected = None
        rx_val_match = None
        rx_unit_match = None

        if drug_can and ex_val is not None and ex_unit is not None and not rxg.empty:
            cand = rxg[rxg.apply(lambda rr: drug_matches(drug_can, rr), axis=1)]
            if not cand.empty:
                for _, rx in cand.iterrows():
                    if doses_equal(ex_val, ex_unit, rx["rx_val_final"], rx["rx_unit_final"]):
                        matched = True
                        drug_expected = (
                            rx["DRUG_NAME_GENERIC"] if isinstance(rx.get("DRUG_NAME_GENERIC"), str)
                            else rx["DRUG"]
                        )
                        rx_val_match = rx["rx_val_final"]
                        rx_unit_match = rx["rx_unit_final"]
                        break

        rows.append({
            "note_row_id": nid,
            "tuple_index": idx,
            "drug_extracted": drug,
            "dose_raw": dose_raw,
            "dose_val_extracted": ex_val,
            "dose_unit_extracted": ex_unit,
            "drug_expected": drug_expected,
            "rx_val_matched": rx_val_match,
            "rx_unit_matched": rx_unit_match,
            "matches_prescription": bool(matched),
        })

results = pd.DataFrame(rows)
matches_df = results[results["matches_prescription"]].reset_index(drop=True)
mismatches_df = results[~results["matches_prescription"]].reset_index(drop=True)

Usable Rx dose/units: 0 (rx_val) -> 14863 (after fallbacks)


In [55]:
matches_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   note_row_id           49 non-null     int64  
 1   tuple_index           49 non-null     int64  
 2   drug_extracted        49 non-null     object 
 3   dose_raw              49 non-null     object 
 4   dose_val_extracted    49 non-null     float64
 5   dose_unit_extracted   49 non-null     object 
 6   drug_expected         49 non-null     object 
 7   rx_val_matched        49 non-null     float64
 8   rx_unit_matched       49 non-null     object 
 9   matches_prescription  49 non-null     bool   
dtypes: bool(1), float64(2), int64(2), object(5)
memory usage: 3.6+ KB


In [56]:
mismatches_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   note_row_id           505 non-null    int64  
 1   tuple_index           505 non-null    int64  
 2   drug_extracted        505 non-null    object 
 3   dose_raw              370 non-null    object 
 4   dose_val_extracted    261 non-null    float64
 5   dose_unit_extracted   261 non-null    object 
 6   drug_expected         0 non-null      object 
 7   rx_val_matched        0 non-null      float64
 8   rx_unit_matched       0 non-null      object 
 9   matches_prescription  505 non-null    bool   
dtypes: bool(1), float64(2), int64(2), object(5)
memory usage: 36.1+ KB


In [57]:
matches_df.head(10)

,note_row_id,tuple_index,drug_extracted,dose_raw,dose_val_extracted,dose_unit_extracted,drug_expected,rx_val_matched,rx_unit_matched,matches_prescription
0,314572,0,fentanyl,25 mcg/hr,25.0,mcg,Fentanyl Patch,25.0,mcg,True
1,314707,0,VANCOMYCIN,1 gram,1.0,g,Vancomycin,1.0,g,True
2,314707,1,VANCOMYCIN,1 gram,1.0,g,Vancomycin,1.0,g,True
3,314739,15,lorazepam,2mg,2.0,mg,Lorazepam,2.0,mg,True
4,314792,0,Methadone,10 mg IV,10.0,mg,Methadone,10.0,mg,True
5,314793,0,Methadone,10 mg IV,10.0,mg,Methadone,10.0,mg,True
6,314795,0,Methadone,10 mg IV,10.0,mg,Methadone,10.0,mg,True
7,314796,0,Methadone,10 mg IV,10.0,mg,Methadone,10.0,mg,True
8,314796,8,Methadone,10 mg IV,10.0,mg,Methadone,10.0,mg,True
9,320483,1,Lorazepam,2mg NG,2.0,mg,Lorazepam,2.0,mg,True


In [58]:
mismatches_df.head(10)

,note_row_id,tuple_index,drug_extracted,dose_raw,dose_val_extracted,dose_unit_extracted,drug_expected,rx_val_matched,rx_unit_matched,matches_prescription
0,314570,0,Fentanyl,50 mcg/hr,50.0,mcg,None,NaN,None,False
1,314570,1,Midazolam,2 mg/hr,2.0,mg,None,NaN,None,False
2,314572,1,midazolam,1 mg/hr,1.0,mg,None,NaN,None,False
3,314572,2,versed,12 mg,12.0,mg,None,NaN,None,False
4,314582,0,vancomycin,1000 mg,1000.0,mg,None,NaN,None,False
5,314582,1,vancomycin,1000 mg,1000.0,mg,None,NaN,None,False
6,314707,2,VANCOMYCIN,-,NaN,None,None,NaN,None,False
7,314707,3,VANCOMYCIN,-,NaN,None,None,NaN,None,False
8,314707,4,VANCOMYCIN,-,NaN,None,None,NaN,None,False
9,314707,5,micrograms per milliliter,None,NaN,None,None,NaN,None,False


In [59]:
mismatches_df_confident = mismatches_df[mismatches_df["drug_expected"].notna()]
mismatches_df_confident.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   note_row_id           0 non-null      int64  
 1   tuple_index           0 non-null      int64  
 2   drug_extracted        0 non-null      object 
 3   dose_raw              0 non-null      object 
 4   dose_val_extracted    0 non-null      float64
 5   dose_unit_extracted   0 non-null      object 
 6   drug_expected         0 non-null      object 
 7   rx_val_matched        0 non-null      float64
 8   rx_unit_matched       0 non-null      object 
 9   matches_prescription  0 non-null      bool   
dtypes: bool(1), float64(2), int64(2), object(5)
memory usage: 0.0+ bytes


This was a failure. I am not sure the idea is bad, but all of my mismatches are instances where I couldnt get a drug from the prescriptions table at all. This tells me I either have bad data or am not properly matching my extractions to the prescriptions table.

In [60]:
conn.close() # sad :(